In [ ]:
!pip install -q pyngrok

In [ ]:
!pip install -q streamlit

In [ ]:
!pip install -q streamlit-ace

In [ ]:
!pip install -q fuzzywuzzy[speedup]

In [ ]:
!streamlit run app.py &>/dev/null&

In [ ]:
from pyngrok import ngrok

In [ ]:
%%writefile app.py
import streamlit
import re
from fuzzywuzzy import process
import json
import requests


def main():

  passportData = requests.get('https://the-icaro.github.io/ago.json')
  namesData = requests.get('https://cspinheiro.github.io/interpol.json')
  
  htmlTemp = """ 
                  <div style="background-color:blue; color:white; padding: 10px; text-align:center; max-height: 100px">
                    <h1>Sistema de Passaporte</h1>
                  </div> 
              """
  streamlit.markdown(htmlTemp, unsafe_allow_html=True)

  ownerName = streamlit.text_input('Nome do Indíviduo:')

  passportNumber = streamlit.text_input('Número do Passaporte:')

  def handleBlockedPassports(passportNumber):

    passport = re.findall('[0-9]+', passportNumber)

    blockedData = passportData.json()['ago']

    for data in blockedData:
      for i, v in data.items():
        if str(passport[0]) == str(v):
          return 'Proibido'
    
    return 'Permitido'

  def handleBlockedPersons(name):

    blockedNamesData = namesData.json()['interpol']

    blocked = []

    for data in blockedNamesData:
      for i, v in data.items():
        blocked.append(v)

    searchList = process.extract(str(name), blocked)

    for i in searchList:
      if i[1] >= 85:
        return 'Proibido'

      return 'Permitido'


  def handleImigrate(name, passportNumber):

    if streamlit.button('Verificar'):

      if not name or not passportNumber:
        streamlit.warning("Preencha todos os campo!")
        return

      passport = handleBlockedPassports(passportNumber)

      person = handleBlockedPersons(name)

      if passport == 'Proibido' or person == 'Proibido':
         streamlit.success("Você não está permitido entrar nesse País")
         return
      
      streamlit.success("Você pode entrar nesse País")
      return

  handleImigrate(ownerName, passportNumber)

if __name__ == '__main__':
  main()

In [ ]:
!streamlit run app.py & npx localtunnel --port 8501